In [25]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [26]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
import json
import time

import folium
import cv2

In [27]:
driver=webdriver.Firefox(executable_path=".\geckodriver.exe")
driver.set_window_position(0, 0)
driver.set_window_size(1024, 768)

C:\Users\Nannu\AppData\Local\Temp/ipykernel_24384/2332002925.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Firefox(executable_path=".\geckodriver.exe")


In [28]:
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [29]:
def get_s2_sr_cld_col(aoi, start_date, end_date,cloud_filter):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    not_water = img.select('SCL').neq(6)
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    img_cloud = add_cloud_bands(img)
    img_cloud_shadow = add_shadow_bands(img_cloud)
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    return img_cloud_shadow.addBands(is_cld_shdw)

def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [33]:
def display_cloud_layers(AOI,col,verbose=False):
    img = col.mosaic()
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    # m.add_ee_layer(cloudmask, {'palette': 'orange'},
    #                'cloudmask', True, 0.5, 9)

    m.add_child(folium.LayerControl())
    if verbose:
        display(m)
    return m

def crop_img(fileName):
    pngName1="./uncropedpngFiles/"+fileName+".png"
    pngName2="./cropedpngFiles/"+fileName+".png"
    img = cv2.imread(pngName1)
    img1=img[0:740,60:960]
    cv2.imwrite(pngName2,img1)

def map_to_png(mapObj, fileName):
    # fileName = 'output'
    
    htmlFile="./htmlFiles/"+fileName+".html"
    mapObj.save(htmlFile)
    driver.get('file://{0}/{1}'.format(os.getcwd(), htmlFile))
    time.sleep(5)
    pngName="./uncropedpngFiles/"+fileName+".png"
    driver.save_screenshot(pngName)
    # driver.quit()
    crop_img(fileName)


In [31]:
cloud_filters=[1,60]
dates=[
    [['2018-01-01','2018-03-02'],['2018-04-01','2018-06-02'],['2018-07-01','2018-09-02'],['2018-10-01','2018-12-02']],
    [['2019-01-01','2019-03-02'],['2019-04-01','2019-06-02'],['2019-07-01','2019-09-02'],['2019-10-01','2019-12-02']],
    [['2020-01-01','2020-03-02'],['2020-04-01','2020-06-02'],['2020-07-01','2020-09-02'],['2020-10-01','2020-12-02']],
    [['2021-01-01','2021-03-02'],['2021-04-01','2021-06-02'],['2021-07-01','2021-09-02'],['2021-10-01','2021-12-02']]
]

In [34]:
fp1=open("./logFiles/CF1.txt","w")
fp2=open("./logFiles/CF60.txt","w")
for x in cloud_filters:
    for y in dates:
        for z in y:
            id=1
            CLOUD_FILTER=x
            START_DATE=z[0]
            END_DATE=z[1]
            AOI = ee.Geometry.Polygon(
                [[[75.8979780320574, 30.24043937830989],
                [75.8979780320574, 30.231429415897423],
                [75.91456738752615, 30.231429415897423],
                [75.91456738752615, 30.24043937830989]]]
            )
            fileName = str(id)+'_CF'+str(CLOUD_FILTER)+'_SD'+str(START_DATE).replace('-',"_")+'_ED'+str(END_DATE).replace('-',"_")
            try:
                s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE,CLOUD_FILTER)
                s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                m = display_cloud_layers(AOI,s2_sr_cld_col_eval_disp)
                map_to_png(m, fileName)
                if CLOUD_FILTER==1:
                    print(fileName,file=fp1)
                else:
                    print(fileName,file=fp2)
            except:
                if CLOUD_FILTER==1:
                    print(fileName + " failed",file=fp1)
                else:
                    print(fileName + " failed",file=fp2)

fp1.close()
fp2.close()

In [22]:
fileName="2"
CLOUD_FILTER = 60
AOI = ee.Geometry.Polygon(
        [[[75.8979780320574, 30.24043937830989],
        [75.8979780320574, 30.231429415897423],
        [75.91456738752615, 30.231429415897423],
        [75.91456738752615, 30.24043937830989]]])
START_DATE = '2020-06-01'
END_DATE = '2020-06-02'


s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE,CLOUD_FILTER)
s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
m = display_cloud_layers(AOI,s2_sr_cld_col_eval_disp,True)
map_to_png(m, fileName)